#  <center> ![Instituto Superior de Engenharia de Lisboa](https://www.isel.pt/media/assets/default/images/logo-isel.png "ISEL")</center>
#  <center> Licenciatura em Informática e Multimédia </center>
## <center> Comunicações e Processamento de Sinais </center>
###  <center> Semestre de Inverno 2021/22 </center>
##  <center> Trabalho Prático 3 </center>

### Turma 31D
### Grupo composto pelos elementos:
* _43033_ - David Nunes
* _45977_ - Eduardo Marques

---
<br/>

A figura representa o esquema típico do processo de envio e recepção de informação usando modulações
digitais.

![fig_trab](esquemaCPS.png)

O 3º trabalho foca o desenvolvimento dos blocos Modulação e desmodulação Digital e com estes é terminado o desenvolvimento de todo o sistema de transmissão digital.

Com o termino deste trabalho deve ser realizado um relatório integrador de todos os blocos.

---
<br/>

## Índice
1. [**Modulação digital QPSK**](#modQPSK)
2. [**Desmodulador QPSK**](#desmodQPSK)
3. [**Canal AWGN e BERt**](#awgnBert)
4. [**Aplicação geral**](#appG)

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from PIL import Image

#################################
### Funções importadas do TP1 ###
#################################

def Quantific(R,Vmax,Qtype):
    delta = (2*Vmax)/(2**R)
    
    if(Qtype == 'midtread'):
        Vq = np.arange(-Vmax, Vmax, delta)
        Iq = np.arange(-Vmax + delta/2, Vmax, delta)
    elif(Qtype == 'midrise'):
        Iq = np.arange(-Vmax, Vmax, delta)
        Vq = np.arange(-Vmax + delta/2, Vmax, delta)
    
    return Vq, Iq

def Quantificador(Rx, Vq, Iq):
    xq = Rx
    iq = np.zeros(len(Rx), dtype=int)
    
    for x in range(len(Iq)):
        if x == 0:
            Rx_x = Rx <= Iq[x]
        else:
            Rx_x = (Rx > Iq[x-1]) & (Rx <= Iq[x])
         
        xq = np.where(Rx_x == True, Vq[x], xq)
        iq = np.where(Rx_x == True, x, iq)
   
    iq = np.where(xq > Iq[len(Iq) - 1], len(Iq)-1, iq)
    xq = np.where(xq > Iq[len(Iq) - 1], Vq[len(Vq) - 1], xq)
      
    return xq, iq

#################################
### Funções importadas do TP2 ###
#################################

def Codific(arr_signal_quantized, R, Vmax, Qtype):
    arr_binary = np.zeros(int(len(arr_signal_quantized)*R), dtype=int)
    aux = 0
    
    if(Qtype == 'midtread'):
        
        Vq, Iq = Quantific(R,Vmax,'midtread')
        for i in range(len(arr_signal_quantized)):
            valor = arr_signal_quantized[i]
            for j in range(2**R):
                if(valor <= Iq[j]):
                    bit = bin(j)[2:]
                    bit2 = str(bit).zfill(R)
                    for k in range(len(bit2)):
                        arr_binary[aux] = bit2[k]
                        aux+=1
                    break                
                if(j == ((2**R-1))):
                    bit = bin(j)[2:]
                    bit2 = str(bit).zfill(R)
                    for k in range(len(bit2)):
                        arr_binary[aux] = bit2[k]
                        aux+=1
                    break
   
    elif(Qtype == 'midrise'):      
        Vq, Iq = Quantific(R,Vmax,'midrise')
        
        for i in range(len(arr_signal_quantized)):
            valor = arr_signal_quantized[i]
            for j in range(2**R):
                if(j < (2**R)-1):
                    if(valor <= Iq[j+1]):
                        bit = bin(j)[2:]
                        bit2 = str(bit).zfill(R)
                        for k in range(len(bit2)):
                            arr_binary[aux] = bit2[k]
                            aux+=1
                        break
                else:
                    bit = bin(j)[2:]
                    bit2 = str(bit).zfill(R)
                    for k in range(len(bit2)):
                        arr_binary[aux] = bit2[k]
                        aux+=1
                    break              
                    
    return arr_binary

def Descodific(arr_binary, R, Vmax, Qtype):    
    arr_signal = np.zeros(int(len(arr_binary)/R), dtype=int)
    
    if(Qtype == 'midtread'):
        Vq, Iq = Quantific(R,Vmax,'midtread')
        for i in range(int(len(arr_binary)/R)):
            array = np.array(arr_binary[i * R : (i+1) * R])
            inteiro = 0
            aux = len(array)
            for j in range(len(array)):
                aux-=1
                if(array[aux]==1):
                    inteiro = inteiro + 2**j
            arr_signal[i] = Vq[inteiro]
            
    elif(Qtype == 'midrise'): 
        Vq, Iq = Quantific(R,Vmax,'midrise')
        for i in range(int(len(arr_binary)/R)):
            array = np.array(arr_binary[i * R : (i+1) * R])
            inteiro = 0
            aux = len(array)
            for j in range(len(array)):
                aux-=1
                if(array[aux]==1):
                    inteiro = inteiro + 2**j
            arr_signal[i] = Vq[inteiro]
        
    return arr_signal

def Hamming_8_4(m, G):   
    c = np.zeros((8,), dtype=int)
    
    for bit in range(len(c)):
        c[bit] = (m[0] & G[0][bit]) ^ (m[1] & G[1][bit]) ^ (m[2] & G[2][bit]) ^ (m[3] & G[3][bit])

    return c

def Detetor(arr_binary, G):
    if (len(arr_binary)%8 is not 0):
        for i in range(8 - (len(arr_binary)%8)):
            arr_binary = np.insert(arr_binary, 0, 0, axis = 0)
            
    P = G[:,4:]
    I = np.array([[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]])
    Ht = np.append(P, I, axis = 0)

    block_array = []
    for i in range(0, len(arr_binary), 8):
        block_array.append(arr_binary[i:i+8])

    msg = []
    for i in range(len(block_array)):
        S = np.dot(block_array[i], Ht)%2

        if not np.array_equal(S, [0, 0, 0, 0]):
            for line in range(len(Ht)):
                if np.array_equal(S, Ht[line]):
                    block_array[i][line] = (1 ^ (block_array[i][line]))

        msg.append(block_array[i][:4])

    msg = np.array(msg).flatten()

    return msg

def BER_pratico(arr_bin_tx, arr_bin_rx):
    erros = np.where( not np.array_equal(arr_bin_rx, arr_bin_tx))
    BER_pratico = len(erros[0])/len(arr_bin_tx)

    return BER_pratico

<a name="modQPSK"></a>
# Modulação digital QPSK

---
Considere as funções criadas nos trabalhos anteriores.

Construa uma função que simule a modulação digital QPSK.
Esta função recebe como parâmetro de entrada um array de bits e retorna um array, simulando o sinal analógico modulado a transmitir. Dado que o sinal modulado é simulado, é necessário que a função ainda receba como parâmetros de entrada o número de pontos por cada símbolo ($P=8$), simulando a amostragem, e a energia média por bit ($E_b$). Admita que a frequência da portadora é tal que cada simbolo contém um período da portadora.

---

Nesta fase do projeto iremos construir uma função que simula a modulação digital QPSK. Esta modulação é uma Modulação Banda Canal onde um sinal modulador (que é uma sequência binária de 0’s e 1’s) vai modular uma portadora sinusoidal. Os parâmetros da portadora são modificados assumindo um
número de estados discretos. Transmite-se informação na forma de um símbolo durante um intervalo de tempo. existem K bits por cada símbolo.
Uma vez que a função que vamos contruir é uma modulação digital QPSK, significa que a cada mudança de símbolo a fase da sinusóide portadora vai mudar. A sua amplitude e frequência permanecem inalteradas.

Nesta primeira fase iremos simular a modulação, onde vai entrar um array de bits e onde vai sair uma sinusóide portadora. O primeiro passo será a adição de x bits caso o tamanho da palavra recebida não seja divisível por k bits, o que iria fazer que que houvesse uma sinusóide incompleta. De seguida calculamos a amplitude que a sinusóide portadora vai ter usando a seguinte formula:

$$\large Eb = (1/2)A^2Tb $$ 

e manipulando a formula temos 

$$\large A = sqrt((Eb * 2)/Tb) $$

A constelação será dividida em $2^k$ partes, onde para cada parte será atribuída k bits e a sua fase, por exemplo se $k=3$ então a constelação será dividida de $2*pi/2^3 = pi/4$ e para o exemplo da palavra [001] será atribuído o valor do bit em decimal a multiplicar pela divisão da constelação, ou seja, a fase de [001] será $pi/4$

In [ ]:
#resposta
def mod_QPSK(bits, P, Eb, k):
    
    """
    8-PSK usando codigo de gray

    aux = np.array([0])
    flag = 0 #usamos as flags para na desmodulação podermos tirar os zeros a mais
    #adsição de bits '0' para o array ficar correto, com grupos de 3 bits
    if(len(bits) % 3 == 1):
        #se [0, , ] então fica [0,0,0]
        bits = np.concatenate((bits,aux))
        bits = np.concatenate((bits,aux))
        flag = 2
    if(len(bits) % 3 == 2):
        #se [0,1, ] então fica [0,1,0]
        bits = np.concatenate((bits,aux))
        flag = 1

    #para cada 3 bits há 8 valores, inicializa tudo a 0
    valPSK = np.zeros(int(len(bits)*(P/3)))
    #P = 1/Rb logo Tb = P
    Tb = P
    #calculo da amplitude
    A = np.sqrt((Eb * 2)/Tb)

    #tempo
    t = np.arange(P)

    #usando o código de gray
    for i in range(int(len(bits)/3)): 
        if(np.array_equal(bits[i * 3 : (i+3)], np.array([0,0,0]))):
            ang = np.pi/4
        elif(np.array_equal(bits[i * 3 : (i+1) * 3], np.array([0,0,1]))):
            ang = np.pi/2
        elif(np.array_equal(bits[i * 3 : (i+1) * 3], np.array([0,1,1]))):
            ang = 3*np.pi/4
        elif(np.array_equal(bits[i * 3 : (i+1) * 3], np.array([0,1,0]))):
            ang = np.pi
        elif(np.array_equal(bits[i * 3 : (i+1) * 3], np.array([1,1,0]))):
            ang = 5*np.pi/4
        elif(np.array_equal(bits[i * 3 : (i+1) * 3], np.array([1,1,1]))):
            ang = 3*np.pi/2
        elif(np.array_equal(bits[i * 3 : (i+1) * 3], np.array([1,0,1]))):
            ang = 7*np.pi/4
        elif(np.array_equal(bits[i * 3 : (i+1) * 3], np.array([1,0,0]))):
            ang = 0 #ou 2pi
        print(ang)
        #sinal do simbolo
        sinal = Anp.cos(2np.pi(t/P) + ang)

        valPSK[iP : (i+1)*P] = sinal

    return valPSK, flag
    """
    
    #inteiro caso array com bits a menos
    m = len(bits) % k
    
    flag = 0 #usamos as flags para na desmodulação podermos tirar os zeros a mais
    #adição de bits '0' para o array ficar correto, com grupos de k bits
    #print(len(bits))
    if(m == 0):
        bits = bits
        flag = 0
    elif(m != 0):
        for i in range(k - m):
            bits = np.concatenate((bits,[0]))
        flag =  k - m #flag = 1 caso tenha que se retirar 1 bit
    #print(len(bits))
    #para cada k bits há 8 valores, inicializa tudo a 0
    valPSK = np.zeros(int(len(bits)*(P/k)))
    #para cada k bits há um angulo da constelação
    ang = np.zeros(int(len(bits)/k))
    #P = 1/Rb logo Tb = P
    Tb = 1/P
    #calculo da amplitude
    A = np.sqrt((Eb * 2)/Tb)  
    #tempo
    t = np.arange(P)
    #usando o código de gray
    for i in range(int(len(bits)/k)): 
        #agrupa x em x bits (ex: i = 2 vai à posição 2*n bits e agrupa os n bits
        #ou seja na palavra[01100100] se for para agrupar de 2 em 2 bits e i=3, vai à posição 6 agrupa o 6 e 7 [00]
        array = np.array(bits[i * k : (i+1) * k])
        
        #converte o inteiro em binario
        inteiro = int("".join(str(x) for x in array), 2)    
        
        #calcula o angulo tento em conta k e o valor do conjunto de bits
        #ang = 2*inteiro*(np.pi/(2**k))
        ang[i] = inteiro*(2*np.pi/(2**k))
        #sinal do simbolo
        sinal = A*np.cos(2*np.pi*(t/P) + ang[i])   

        valPSK[i*P : (i+1)*P] = sinal
        
    return valPSK, flag, ang

bits = [0,0,0,0,1,0]
Eb = 2 * 10e-6
#numero de bits por amostra
k = 3
#val, flag = mod_QPSK(bits, 100, Eb)
val, flag, angulo = mod_QPSK(bits, 80, Eb, k)
plt.plot(val)
plt.show()

<a name="desmodQPSK"></a>
# Desmodulador QPSK

---
Construa uma função que simule, no receptor, um filtro adaptado (desmodulação).
Este deve receber um array com o sinal QPSK com ruído e deve retornar um array com a sequência binária correspondente.

---

Iremos agora construir o desmodulador que recebe um array com o sinal QPSK com ruído e deve retornar um array com a sequência binária correspondente. Para tal teremos que calcular os valores de decisão que facilmente o fazemos somando qualquer dois ângulos de duas regiões adjacentes e dividindo por 2 fazendo assim a média. A diferença entre o valor máximo e mínimo das regiões de decisão é a mesma diferença entre duas regiões adjacentes da constelação.

De seguida comparamos os valores e atribuímos uma região e, por sua vez, o conjunto de bits equivalente à região onde se encontra.

Por fim, como podem ter sido adicionados x bits na modulação para o caso do tamanho da palavra recebida não seja divisível por k bits, iremos retirar os x bits para a palavra na saída ter o mesmo tamanho que a palavra inicial.

In [ ]:
#resposta
def desMod_QPSK(val, flag, P, k, angulo):
    
    #nº de conjuntos de k bits que há
    n = len(val) / P
    #variável aux para meter os bits no array, começa no bit de menor peso
    aux1 = 0
    #para cada n conjuntos cada um tem k bits
    bitsDesMod = np.zeros(int(n*k), dtype=int)
    
    valconst = np.zeros(int(2**k))
    for a in range(2**k):
        #valores da constelação
        valconst[a] = (2*np.pi/2**k)*a
    
    #região de decisão iniciais(max e min)
    regDMax = (valconst[0] + valconst[1])/2
    regDMin = -regDMax
    
    for b in range(len(angulo)):       
        #se o angulo estiver entre 2pi e 2pi - valor de decisão tira 2pi senão não faz
        if(angulo[b] > (2*np.pi - regDMax) and angulo[b] <= 2*np.pi):
            angulo[b] = angulo[b] - 2*np.pi
        for c in range(2**k):
            #a cada ciclo vai adicionando o valor de uma "fatia" da constelação
            maxi = regDMax + (2*np.pi/2**k)*c
            mini = regDMin + (2*np.pi/2**k)*c
            
            #quando o angulo estiver dentro dos valores substitui o b pelo equivalente em binario
            if(angulo[b] <= maxi and angulo[b] > mini):
                bit = bin(c)[2:]
                #se o bit for 1 o zfill(k) k=3 adiciona 2 zeros atras do 1 001
                bit2 = str(bit).zfill(k)   
                for d in range(len(bit2)):
                    bitsDesMod[aux1] = bit2[d]                  
                    aux1+=1    
                    
    #tira os bits a mais que foram usados na modulação 
    if(flag != 0):
        bitsDesMod = bitsDesMod[0:len(bitsDesMod) - flag]
          
    return bitsDesMod

bits = [1,0,1,1]
Eb = 2 * 10e-6
#numero de bits por amostra
k = 10

#val, flag = mod_QPSK(bits, 100, Eb)
val, flag, angulo = mod_QPSK(bits, 8, Eb, k)
bitsf = desMod_QPSK(val, flag, 8, k, angulo)
print(bits)
print(bitsf)

<a name="awgnBert"></a>
# Canal AWGN e BERt

---
Construa uma função que simule um canal AWGN.
Esta função recebe o sinal modulado, adiciona ruído (com uma determinada potência $\sigma_n$)
e devolve o sinal com ruído num array.
A função deve ter como parâmetro de entrada a potência do ruído a adicionar

---

In [ ]:
#resposta
def canal_AWGN(sinal_in, pot_noise):
    sinal_out = sinal_in + np.sqrt(pot_noise) * np.random.randn(len(sinal_in))
    
    return sinal_out

---
Construa uma função que calcule o BER teórico para a modulação implementada.

---

Este BER teórico aqui desenvolvido trata-se da probabilidade de erro de bit em cada nível lógico calculada através das respetivas probabilidades à priori, ou seja, trata-se da soma das probabilidades de enviar um bit de nível lógico "1" mas descodificar como se este se tratasse de um bite de nível lógico "0", e vice-versa. Estas probabilidades de errar os bits ao nível lógico "0" ou "1" são iguais, pelo que o BER é obtido em função da função complementar de erro.

In [ ]:
#AWNG
def BER_teorico(Eb, No, k):
    #resposta
    BER_t = (1/k) * math.erfc(np.sqrt(k * np.sin(np.pi/2**k)**2 * (Eb/No)))
    return BER_t


No = 10e-6     #densidade espetral
Eb = 2 * 10e-6 #energia média por bit
k = 3          #numero de bits
ber = BER_teorico(Eb, No, k)
print(ber)

<a name="appG"></a>
# Aplicação geral

---
Avalie todo o sistema de emissão e receção. Para isso deve medir experimentalmente e comparar, sempre que possível, com os valores teóricos, os seguintes índices:

    A SNR no canal e SNR na recepção;
    
    BER antes e depois da correcção de erros;
    
    Apresente a constelação antes e depois do sinal passar pelo canal.

Para o efeito deve atribuir diferentes valores aos parâmetros do sistema e avalia-lo com e sem o modulo de correcção de erros.

Use a imagem fornecida anteriormente para experimentar o sistema com todos os blocos desenvolvidos

---

O primeiro passo na emissão da imagem é a quantificação da mesma, de forma a, de seguida, efetuar a sua codificação. Abaixo podemos observar a imagem quantificada.

In [ ]:
R = 4  # 3,4,6,8
Vmax = 255
img = Image.open("lena_gray.tif")
img_array = np.array(img)
shp = img_array.shape
Vq, Iq = Quantific(R,Vmax,'midrise')

for i in range(len(img_array)):
    y = Quantificador(img_array[i],Vq,Iq)
    img_array[i] = y[0]
    
img_quant = Image.fromarray(img_array)
display(img_quant)

De seguida efetua-se então a codificação, que consiste basicamente em tornar o sinal de imagem mais apropriado para a transmissão através do canal, neste caso, convertemos os sinal quantizado num array de bits correspondente aos valores que pretendemos quantificar. Este array é depois convertido num array unidimensional de forma a ser enviado para a função Hamming(8,4) que pega em blocos de 4 bits do nosso array e introduz bits de paridade de forma a permitir encontrar e corrigir erros que possam ocorrer durante a transmissão.

In [ ]:
img_codific = []
G_matrix=np.array([[1,0,0,0,1,1,0,1],[0,1,0,0,1,0,1,1],[0,0,1,0,0,1,1,1],[0,0,0,1,1,1,1,0]])
bin_array = []
code_msg = []

for i in range(len(img_array)):
    c = Codific(img_array[i],R,Vmax,'midrise')
    img_codific.append(c)

img_codific = np.array(img_codific).flatten()

for i in range(0, len(img_codific), 4):
    bin_array.append(img_codific[i:i+4])

bin_array = np.array(bin_array)

for i in range(len(bin_array)):
    code_msg.append(np.array(Hamming_8_4(bin_array[i], G_matrix)))
    
code_msg = np.array(code_msg).flatten()

É neste momento que enviamos a mensagem de código obtida a partir da função de Hamming, logo após simular a modulação digital QPSK. No canal é introduzido barulho, que provoca erros nos bits, alterando o seu valor.

In [ ]:
P = 8
Eb = 4
k = 4
noise = 100

imgQPSK, flag, ang = mod_QPSK(code_msg, P, Eb, k)
imgQPSK = canal_AWGN(imgQPSK, noise)

In [ ]:
print("BER antes de corrigir erros: " + str(BER_pratico(code_msg, imgQPSK)))

Estes erros podem ser detetados e corrigidos de seguida, quando o sinal desmodulado é fornecido à função detetora e corretora de erros. Caso o "noise" seja muito alto, a imagem pode mesmo ficar irrecuperável, notando-se um padrão randomizado de bits. Abaixo podemos observar a imagem obtida para os valores fornecidos.

In [ ]:
bits_desmod = desMod_QPSK(imgQPSK, flag, P, k, ang)

msg = Detetor(bits_desmod, G_matrix)

print("BER depois de corrigir erros: " + str(BER_pratico(code_msg, msg)))

In [ ]:
d = Descodific(msg,R,Vmax,'midrise')

img_final = Image.fromarray((np.array(d, dtype='uint8')).reshape(shp))
display(img_final)